# Library

In [1]:
!pip install einops
!pip install timm

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as T
import pandas as pd
import numpy as np
from einops import rearrange # for changing channels

from glob import glob
import json
import csv
import cv2 # for image load
from PIL import Image
import albumentations as A
import albumentations.pytorch
import torchvision.transforms as transforms
import os
import random
import plotly.express as px # for graph

import timm # for pretrained models
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

from sklearn.metrics import f1_score # for f1 score

from tqdm import tqdm
import time
from time import sleep

# Get Dataset

In [ ]:
!unzip "/content/drive/MyDrive/3d/TestSet.zip" -d "/content/"

FAKE_IMG_PATH = '/content/TestSet/'

In [ ]:
!mkdir "tars"
!tar -xvf "/content/drive/MyDrive/3d/ILSVRC2012_img_train_t3.tar" -C  "/content/tars"

In [5]:
!mkdir "REAL"

dir_list = os.listdir("/content/tars")

for dir in dir_list:
  os.system(f"tar -xvf  /content/tars/{dir} -C  /content/REAL")

REAL_IMG_PATH = '/content/REAL/'

mkdir: cannot create directory ‘REAL’: File exists


# Env

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

random.seed(1234)

model_list = {
    "convnext_base_in22ft1k" : {
        "model" : "convnext_base_in22ft1k", 
        "input_size" : (224, 224),
        "classifier_in_feature" : 1024
        }
}

optimizer_list = ["adam", "sgd"]

model_name = "convnext_base_in22ft1k"

CONFIG = {
    "batch_size" : 16,
    "epoch" : 200,
    "learning_rate" : 1e-4,
    "input_size" : model_list[model_name]["input_size"],
    "backbone" : model_list[model_name]["model"],
    "classifier_in_feature": model_list[model_name]["classifier_in_feature"],
    "device" : device,
    "patience" : 10,
    "optimizer" : optimizer_list[0],
    "input_norm_mean" : IMAGENET_DEFAULT_MEAN,
    "input_norm_std" : IMAGENET_DEFAULT_STD
}

batch_size = CONFIG["batch_size"]
epochs = CONFIG["epoch"]
learning_rate = CONFIG["learning_rate"]
input_size = CONFIG["input_size"]
backbone = CONFIG["backbone"]
classifier_in_feature = CONFIG["classifier_in_feature"]
device = CONFIG["device"]
patience = CONFIG["patience"]
optimizer = CONFIG["optimizer"]
input_norm_mean = CONFIG["input_norm_mean"]
input_norm_std = CONFIG["input_norm_std"]

Using cuda device


# Read the Data

In [14]:
class CustomDataset(Dataset):
    def __init__(self, FAKE_path_label, REAL_path_label, input_size, transform=None, isTest=False):
        if isTest:
            self.FAKE_path_label = FAKE_path_label
            self.REAL_path_label = REAL_path_label
            self.input_size = input_size
            self.transform = transform
            self.isTest = isTest
            self.img_list = []
            
            for i in range(len(FAKE_path_label)):
                img_path = FAKE_path_label[i][0]
                # image = cv2.imread(img_path)
                image = Image.open(img_path)
                if self.transform:
                    image = self.transform(image=np.array(image))['image']
                self.img_list.append((image, FAKE_path_label[1]))

            for i in range(len(REAL_path_label)):
                img_path = REAL_path_label[i][0]
                # image = cv2.imread(img_path)
                image = Image.open(img_path)
                if self.transform:
                    image = self.transform(image=np.array(image))['image']
                self.img_list.append((image, 1))
        else:
            pass

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        image, label = self.img_list[idx]

        return image, label #label type: int

In [18]:
FAKE_path_label = []
REAL_path_label = []

# Get Fake Data
with open(FAKE_IMG_PATH+"operations.csv", "r") as operations_csv:
    i = csv.reader(operations_csv, delimiter=',')
    for j in i:
        if j[6] == "FALSE":
            FAKE_path_label.append((FAKE_IMG_PATH+j[0], 0)) # 0 for false
FAKE_path_label = FAKE_path_label[1:]
print(FAKE_path_label[0])
print(len(FAKE_path_label))

# Get Real Data
REAL_path_label = [(os.path.join(REAL_IMG_PATH, path), 1) for path in os.listdir(REAL_IMG_PATH)]
print(REAL_path_label[0])
print(len(REAL_path_label))

A_transform = A.Compose([
    A.PadIfNeeded(*input_size),
    albumentations.augmentations.crops.transforms.CenterCrop(*input_size),
    # albumentations.augmentations.transforms.Normalize(
    #     mean=input_norm_mean, 
    #     std=input_norm_std
    #     ),
    albumentations.pytorch.transforms.ToTensorV2()
])

FAKE_path_label = random.sample(FAKE_path_label, k=1000)
REAL_path_label = random.sample(REAL_path_label, k=1000)

test_dataset = CustomDataset(FAKE_path_label, REAL_path_label, A_transform, isTest=True)
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

('/content/TestSet/biggan_256/biggan_002_230631.png', 0)
15999
('/content/REAL/n02106382_2021.JPEG', 1)
20580


# Model

In [11]:
class Model(nn.Module):
    def __init__(self, backbone, classifier_in_feature, pretrained=True):
        super(Model, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.classifier = nn.Linear(
                in_features=classifier_in_feature,
                out_features=2
            )
        
        for param in self.backbone.parameters():
            param.requires_grad=False

    def forward(self, images):
        output = self.backbone(images) # bs 1 1000
        output = self.classifier(output)
        return output


model = Model(backbone, classifier_in_feature, pretrained=True)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()

if optimizer == "adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
elif optimizer == "sgd":
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Test

In [19]:
@torch.no_grad()
def test_model(device, model, test_loader):
    pred_ans = []

    model.eval()
    model = model.to(device)

    for batch, (x, y) in enumerate(tqdm(test_loader)):
        x, y = x.to(device), y.to(device)

        outputs = model(x)
        
        pred_ans += [(output, y) for output in outputs]


    return pred_ans


In [20]:
correct = [0,0,0,0] # 1,1 0,0 1,0 0,1

pred_ans = test_model(device, model, test_dataloader)

for (x, y) in pred_ans:
    if x == 1 and y == 1:
        correct[0] += 1
    elif x == 0 and y == 0:
        correct[1] += 1
    elif x == 1 and y == 0:
        correct[2] += 1
    elif x == 0 and y == 1:
        correct[3] += 1
    else:
        print(f"x:{x}, x type:{type(x)}, y:{y}, y type:{type(y)}")


print(pred_ans)
print(correct)

  0%|          | 0/125 [00:00<?, ?it/s]


TypeError: ignored